Market basket analysis - association_rules

In [1]:
# Importing libraries
#!pip install mlxtend

import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# Importing data set

trans_part1 = pd.read_csv("E:\\Training data\\PGDS- EDA\\Batch 6\\instacart_2017_05_01\\order_products__part1.csv")
trans_part2 = pd.read_csv("E:\\Training data\\PGDS- EDA\\Batch 6\\instacart_2017_05_01\\order_products__part2.csv")
orderDf = pd.read_csv("E:\\Training data\\PGDS- EDA\\Batch 6\\instacart_2017_05_01\\orders.csv")
productDf = pd.read_csv("E:\\Training data\\PGDS- EDA\\Batch 6\\instacart_2017_05_01\\products.csv")

In [3]:
# we will merge the trans_part1 and trans_part2 dataset to get the complete order dataset.

trans_df = trans_part2.append(trans_part1,ignore_index = True)

trans_df.shape

(33819106, 4)

STEP 1 : we will find the most frequently purchased products - May be top 20 as on whole data iterations will take long time. Also for all those frequently purchased products we will also get the product name

In [4]:
# Lets find the frequency of purchase for each product by counting the order_id for each product_id

productCountDf = trans_part2.groupby(["product_id"])["order_id"].count().reset_index()

# Lets rename the column name "order_id" to freq

productCountDf = productCountDf.rename(columns = {'order_id' : 'Freq'})

# We will be interseted to make association rules only for top 100 products

productCountDf_top_20 = productCountDf.sort_values(by='Freq', ascending= False).head(20)

productCountDf_top_20.head()


,product_id,Freq
24848,24852,472565
13172,13176,379450
21133,21137,264683
21899,21903,241921
47198,47209,213584


In [5]:
# Getting the product name for each of these product ID

newproductCountDf = pd.merge(productCountDf_top_20, productDf, left_on='product_id', right_on='product_id', how='left')

# And I am intersted only in the column of product ID and product name from the merged data

newDf = newproductCountDf[['product_id','product_name']]

newDf.head()


,product_id,product_name
0,24852,Banana
1,13176,Bag of Organic Bananas
2,21137,Organic Strawberries
3,21903,Organic Baby Spinach
4,47209,Organic Hass Avocado


STEP 2: Now we will filter the orders data frame and get orders containting the top 20 frequently purchased products.

In [6]:
# Step 1: lets make the list of all the product id in top 20

productId= newDf[["product_id"]]

In [7]:
# Step 2: We will iterate through our trans_df to filter all the orders contining the product IDs in the above list

df = trans_df[0:0]

for i in range(0,19):
    pId = productId.iloc[i]['product_id'] 
    stDf = trans_df[trans_df.product_id == pId ]
    df = df.append(stDf,ignore_index = False)

In [8]:
# Lets also get the product name in data frame df

df = pd.merge(df, newDf, left_on='product_id', right_on='product_id', how='left')

df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,10,24852,1,1,Banana
1,20,24852,6,0,Banana
2,22,24852,3,1,Banana
3,26,24852,2,1,Banana
4,52,24852,2,1,Banana


STEP 3: Now we need to consolidate the items into 1 transaction per row with each product 1 hot encoded(Type of DTM). Each row will represent an order and each column will represent product_id. 

If the cell value is '1' say (i,j) then ith order contains jth product.

In [9]:
# Step 1: Find the freq of each product in each order by groupby on order id and product name

basket = df.groupby(['order_id', 'product_name']).size().reset_index()

In [10]:
basket.head()

,order_id,product_name,0
0,1,Bag of Organic Bananas,1
1,1,Cucumber Kirby,1
2,1,Organic Hass Avocado,1
3,3,Organic Baby Spinach,1
4,5,Bag of Organic Bananas,1


In [11]:
# Step 2: This is the most chellanging step. It will take time. We need to convert this long data to wide data
# Where index will be order id and column will be product name

basket1 = basket.pivot('order_id', 'product_name',  0)

In [12]:
basket1.head()

product_name,Apple Honeycrisp Organic,Bag of Organic Bananas,Banana,Cucumber Kirby,Large Lemon,Limes,Organic Avocado,Organic Baby Spinach,Organic Blueberries,Organic Fuji Apple,Organic Garlic,Organic Hass Avocado,Organic Lemon,Organic Raspberries,Organic Strawberries,Organic Whole Milk,Organic Yellow Onion,Organic Zucchini,Strawberries
order_id,,,,,,,,,,,,,,,,,,,
1,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [13]:
# This is another time taking step
# Orders in which the product is not avaiable, those cells will take value of NaN. Thus we need to replace those cells by 0

basket2 = basket1.fillna(0)

In [14]:
basket2.head()

product_name,Apple Honeycrisp Organic,Bag of Organic Bananas,Banana,Cucumber Kirby,Large Lemon,Limes,Organic Avocado,Organic Baby Spinach,Organic Blueberries,Organic Fuji Apple,Organic Garlic,Organic Hass Avocado,Organic Lemon,Organic Raspberries,Organic Strawberries,Organic Whole Milk,Organic Yellow Onion,Organic Zucchini,Strawberries
order_id,,,,,,,,,,,,,,,,,,,
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


STEP 4: Now that the data is structured properly, we can generate frequent item sets with some cut-off on support

In [15]:
# Build up the frequent items

frequent_itemsets = apriori(basket2, min_support=0.01, use_colnames=True)

STEP 5: The final step is to generate the rules with their corresponding support, confidence and lift:

In [17]:
# Create the rules

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [21]:
# We can reduce the rules for lift greater than 2 and confidence more than 0.2

rules_final = rules[ (rules['lift'] > 1) & (rules['confidence'] >= 0.2) ]

rules_final


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(Apple Honeycrisp Organic),(Bag of Organic Bananas),0.048822,0.220935,0.013641,0.279391,1.264582,0.002854,1.081120
3,(Organic Hass Avocado),(Bag of Organic Bananas),0.123565,0.220935,0.036229,0.293199,1.327083,0.008929,1.102241
5,(Organic Lemon),(Bag of Organic Bananas),0.051048,0.220935,0.013119,0.256983,1.163162,0.001840,1.048516
7,(Organic Raspberries),(Bag of Organic Bananas),0.079776,0.220935,0.023654,0.296508,1.342060,0.006029,1.107426
9,(Organic Strawberries),(Bag of Organic Bananas),0.154166,0.220935,0.036196,0.234787,1.062697,0.002136,1.018102
11,(Cucumber Kirby),(Banana),0.055791,0.274842,0.018372,0.329296,1.198125,0.003038,1.081188
12,(Organic Avocado),(Banana),0.103060,0.274842,0.031110,0.301866,1.098325,0.002785,1.038709
14,(Organic Fuji Apple),(Banana),0.051965,0.274842,0.019666,0.378441,1.376938,0.005383,1.166675
16,(Strawberries),(Banana),0.083604,0.274842,0.024156,0.288936,1.051278,0.001178,1.019820
42,(Organic Blueberries),(Organic Strawberries),0.058755,0.154166,0.014000,0.238274,1.545572,0.004942,1.110418
